In [178]:
import os
import sys
import numpy as np

import logging
import sys
from datetime import datetime
import time

from multiprocessing import Pool



In [179]:
enableDebugLog = False
enableResultLog = True

In [180]:

def createDirectory(path):
    path = path.split(os.sep)
    currentDir = '.'
    for i in path:
        currentDir += os.sep + i
        try:
            os.mkdir(currentDir)
        except:
            continue


In [181]:
def createOrAppendFile(filePath, clean = False):
    mode = ''
    if os.path.isfile(filePath):
        if clean: mode = 'w'
        else: mode = 'a'
    else: mode = 'x'
    return open(filePath, mode)
        # file.close()


In [182]:

logDir = './log/' + datetime.today().strftime('%Y-%m-%d')
logFileName = datetime.today().strftime('%H-%M')
logFilePath = f'{logDir}/{logFileName}.log'
createDirectory(logDir)
fileHandler = createOrAppendFile(logFilePath)

logFileName


'20-27'

In [183]:
def writeToFile(msg, enable, handler=fileHandler):
    if enable:
        handler.write(f'\n{msg}')
        handler.flush()
def printDetail(msg, obj):
    print(f'{msg}: {obj}')


In [184]:
writeToFile('==================\n\n\nstart logging', enableDebugLog)


In [185]:
network = {}
allNode = set()


In [186]:
##################################################
# Build new strictly increasing Ids
##################################################
edgeCounting = 0
with open("./web-Google.txt") as infile:
    for line in infile:
        if line.startswith('#'):
            continue

        line = line.strip('\\s')
        try:
            webId, linkedTo = line.split()
        except:
            print(f'fail to parse data:\n {line}')
            continue
        webId = int(webId)
        linkedTo = int(linkedTo)
        allNode.add(webId)
        allNode.add(linkedTo)

        edgeCounting +=1
        
        # if webId not in network:
        #     network[webId] = [linkedTo]
        # else:
        #     network[webId].append(linkedTo)
infile.close()


fail to parse data:
 



In [187]:
edgeCounting


7

In [188]:
allNode = sorted(allNode)


In [189]:
len(allNode)


4

In [190]:
printDetail('network\'s last node', allNode[-1])
printDetail('network\'s length', len(allNode))
dif = '%.2f' % ((allNode[-1] - len(allNode))/allNode[-1]*100)
printDetail('dif', dif + '%')


network's last node: 3
network's length: 4
dif: -33.33%


In [191]:
# key: old index, value: new index
nodeReIndex = {}
i = 0
for value in allNode:
    nodeReIndex[value] = i
    i+=1
    

In [192]:
# nodeReIndex[916425]


In [193]:
##################################################
# Lookup new Id based on old Id
##################################################
def convertSingleIdToNewIndex(id, newIndex):
    return newIndex[id]

oldIds = list(nodeReIndex.keys())
newIds = list(nodeReIndex.values())

##################################################
# Get old Id based on new Id
##################################################
def getOldId(newId):
    return oldIds[newIds.index(newId)]
    

In [194]:
# getOldId(875710)


In [195]:
##################################################
# Build the "FROM" adjacency data
##################################################
network = {}
with open("./web-Google.txt") as infile:
    for line in infile:
        if line.startswith('#'):
            continue

        line = line.strip('\\s')
        try:
            webId, linkedTo = line.split()
        except:
            print(f'fail to parse data:\n {line}')
            continue
        webId = convertSingleIdToNewIndex(int(webId), nodeReIndex)
        linkedTo = convertSingleIdToNewIndex(int(linkedTo), nodeReIndex)
        # allNode.add(webId)
        # allNode.add(linkedTo)

        if webId not in network:
            network[webId] = [linkedTo]
        else:
            network[webId].append(linkedTo)
infile.close()


fail to parse data:
 



In [196]:
network[0]


[1, 2, 3]

In [197]:
# networkMatrix = np.zeros((len(nodeReIndex),len(nodeReIndex)))
networkMatrix = np.zeros((1,len(nodeReIndex)))

printDetail('len(nodeReIndex)', len(nodeReIndex))
printDetail('sys.getsizeof(networkMatrix)(MB)', sys.getsizeof(networkMatrix)/10**6)

# sys.getsizeof(networkMatrix)


len(nodeReIndex): 4
sys.getsizeof(networkMatrix)(MB): 0.000152


#### try convert to full matrix and store in file
#### they were too big

#### only update index to new index
#### SKIP the 0s

In [198]:
# d = 0.85
d = 1
eOverN = 1/len(allNode)


def calcRankProcess(calcData):

    nodeId = calcData[0]
    toNode_coef = calcData[1]
    rank = calcData[2]
    writeToFile(f'> calcRankProcess: {nodeId}\n\t\t{toNode_coef}', enableDebugLog)
    result = calcRank(toNode_coef, rank)
    # with runnerCounter.get_lock():
    #     runnerCounter.value -= 1

    
    writeToFile(f'> calcRankProcess: node {nodeId}\'s new rank : {result}', enableDebugLog)
    # print(f'calc rank for node {nodeId} : {result}')
    return result



def calcRank(linkingCoef, currentRankFull):
    return d*calcRankDirectLink(linkingCoef, currentRankFull) + (1-d)*eOverN
#     return calcRankDirectLink(linkingCoef, currentRankFull)

def calcRankDirectLink(linkingCoef:dict, currentRankFull):
    sum = 0

    try:
        for k, v in linkingCoef.items():
            sum += v*currentRankFull[k]
    except:
        writeToFile(f'ERROR!!!! linking coef = {linkingCoef.items()}', True)
    return sum


In [199]:
network_toNode = dict()

##################################################
# Build the "TO" adjacency data
##################################################
def build_networkToNode_coef(network_fromNode_updatedIndex: dict()):
    result = {}
    for id, linkTo in network_fromNode_updatedIndex.items():
        coef = 1/len(linkTo)
        for toNode in linkTo:
            try:
                result[toNode][id] = coef
            except:
                result[toNode] = {id: coef}
    return result


network_toNode = build_networkToNode_coef(network)


In [200]:
network_toNode[0]


{1: 0.5}

In [201]:
def buildData(currentRank):
    for oldId, newId in nodeReIndex.items():
        writeToFile(f'\t> calculating for node {newId}',enableDebugLog)
        try:
            data.append([newId, network_toNode[newId], currentRank])
        except Exception as e:
            data.append([newId,{},currentRank])
            checkAvailable = newId in network_toNode
            writeToFile(f'\t\t{e}\n\t\t{newId} in network_toNode : {checkAvailable}', True)
def updateData(newRank):
    for row in data:
        del row[2]
        row.append(newRank)


In [202]:


# data =[]
# currentRank = [1/len(allNode)]*len(allNode)
# buildData(currentRank)


In [203]:
print(f'sys.getsizeof(data)(MB)= {sys.getsizeof(data)/10**6}')


sys.getsizeof(data)(MB)= 8.8e-05


In [204]:
# nodeReIndex[916097]


In [205]:
resultDir = logDir + '/result'
createDirectory(resultDir)


In [206]:
pool = Pool(100)


def calcIterator(iterNo, currentRank, network_toNode):
    resultFileName = f'{resultDir}/{iterNo}.log'
#     print(resultFileName)
    resultFileHandler = createOrAppendFile(resultFileName)
    
    updateData(currentRank)
    
    
    result = pool.map(calcRankProcess, data)
    for nodeRanking in result:
        writeToFile(nodeRanking, True, resultFileHandler)
    resultFileHandler.close()
    return result


In [207]:
time.time()


1624013821.787253

In [208]:
%%time

# N = 50
N = 50


data =[]
currentRank = [1/len(allNode)]*len(allNode)
buildData(currentRank)

startingTime = None
for iterNo in range(N):
    startingTime = time.time()
    currentRank = calcIterator(iterNo, currentRank, network_toNode)
    print(f'finish iter #{iterNo} in {time.time() - startingTime}\ncurrent rank:{currentRank}\n')


finish iter #0 in 0.0010442733764648438
current rank:[0.125, 0.20833333333333331, 0.20833333333333331, 0.20833333333333331]
finish iter #1 in 0.0008459091186523438
current rank:[0.10416666666666666, 0.14583333333333331, 0.14583333333333331, 0.14583333333333331]
finish iter #2 in 0.0008411407470703125
current rank:[0.07291666666666666, 0.10763888888888887, 0.10763888888888887, 0.10763888888888887]
finish iter #3 in 0.0012161731719970703
current rank:[0.053819444444444434, 0.07812499999999999, 0.07812499999999999, 0.07812499999999999]
finish iter #4 in 0.0006811618804931641
current rank:[0.03906249999999999, 0.057002314814814804, 0.057002314814814804, 0.057002314814814804]
finish iter #5 in 0.002213001251220703
current rank:[0.028501157407407402, 0.041521990740740734, 0.041521990740740734, 0.041521990740740734]
finish iter #6 in 0.001154184341430664
current rank:[0.020760995370370367, 0.0302613811728395, 0.0302613811728395, 0.0302613811728395]
finish iter #7 in 0.000606536865234375
curre

In [209]:
currentRank


[2.554074167615562e-08,
 3.722376927103125e-08,
 3.722376927103125e-08,
 3.722376927103125e-08]

In [210]:
finalResult = currentRank


In [211]:
top10rank = list(sorted(finalResult, reverse=True)[:10])


In [212]:
top10rank


[3.722376927103125e-08,
 3.722376927103125e-08,
 3.722376927103125e-08,
 2.554074167615562e-08]

In [213]:
top10rank.count(0.14303660180461966)


0

In [214]:
for rank in top10rank:
    if top10rank.count(rank) > 1:
        top10rank.remove(rank)


In [215]:
##################################################
# Get node Id by comparing top ranking and current ranking
# if multiple nodes has same ranking, return all of them
##################################################
def getNodeIdByRank(rank, fullRank=currentRank):
    return [i for i, x in enumerate(fullRank) if x == rank]


In [216]:
top10 = []
for x in top10rank:
    for y in getNodeIdByRank(x):
        top10.append(getOldId(y))
top10


[1, 2, 3, 0]

In [217]:
testlist = [0,1,2,1,3]
for rank in testlist:
    if testlist.count(rank) > 1:
        testlist.remove(rank)
testlist

[0, 2, 1, 3]